[Reference](https://medium.com/mlearning-ai/football-analytics-101-how-to-scrape-data-from-understat-com-74ea1cb0da17)

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
#scrape a single game shots. We will allow users to enter a base url
base_url = 'https://understat.com/match/'
match = str(input('Please enter the match id: '))
url = base_url+match

Please enter the match id: 16754


In [3]:
#Use requests to get the webpage and BeautifulSoup to parse the page
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

In [4]:
#get only the shotsData
strings = scripts[1].string

In [5]:
# strip unnecessary symbols and get only JSON data 
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
data = json.loads(json_data)

data

{'h': [{'id': '477932',
   'minute': '2',
   'result': 'MissedShots',
   'X': '0.955999984741211',
   'Y': '0.605999984741211',
   'xG': '0.0635618269443512',
   'player': 'Fernandinho',
   'h_a': 'h',
   'player_id': '614',
   'situation': 'SetPiece',
   'season': '2021',
   'shotType': 'Head',
   'match_id': '16754',
   'h_team': 'Manchester City',
   'a_team': 'Aston Villa',
   'h_goals': '3',
   'a_goals': '2',
   'date': '2022-05-22 15:00:00',
   'player_assisted': 'Kevin De Bruyne',
   'lastAction': 'Cross'},
  {'id': '477933',
   'minute': '4',
   'result': 'BlockedShot',
   'X': '0.855',
   'Y': '0.649000015258789',
   'xG': '0.06113557890057564',
   'player': 'João Cancelo',
   'h_a': 'h',
   'player_id': '2379',
   'situation': 'OpenPlay',
   'season': '2021',
   'shotType': 'RightFoot',
   'match_id': '16754',
   'h_team': 'Manchester City',
   'a_team': 'Aston Villa',
   'h_goals': '3',
   'a_goals': '2',
   'date': '2022-05-22 15:00:00',
   'player_assisted': 'Phil Foden',

In [6]:
x = []
y = []
xG = []
result = []
team = []
data_away = data['a']
data_home = data['h']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])

In [7]:
col_names = ['x','y','xG','result','team']
df = pd.DataFrame([x,y,xG,result,team],index=col_names)
df = df.T

df

,x,y,xG,result,team
0,0.955999984741211,0.605999984741211,0.0635618269443512,MissedShots,Manchester City
1,0.855,0.649000015258789,0.06113557890057564,BlockedShot,Manchester City
2,0.8569999694824219,0.43099998474121093,0.09564501792192459,BlockedShot,Manchester City
3,0.8919999694824219,0.74,0.032729268074035645,BlockedShot,Manchester City
4,0.845999984741211,0.6140000152587891,0.06224839761853218,MissedShots,Manchester City
5,0.9680000305175781,0.34700000762939454,0.05484689027070999,BlockedShot,Manchester City
6,0.75,0.7069999694824218,0.01756650023162365,BlockedShot,Manchester City
7,0.7859999847412109,0.610999984741211,0.025742145255208015,BlockedShot,Manchester City
8,0.9180000305175782,0.5640000152587891,0.06785253435373306,SavedShot,Manchester City
9,0.8980000305175782,0.425,0.050726842135190964,MissedShots,Manchester City
